In [1]:
''' Advance Python Project Assignment: '''

' Advance Python Project Assignment: '

In [ ]:
# 1. Build a Flask app that scrapes data from multiple websites and displays it on your site.
# You can try to scrap websites like youtube , amazon and show data on output pages and deploy it on cloud platform .

'''
a Flask app that scrapes data from multiple websites (such as YouTube and Amazon) and displays it on 
site using Python libraries like Requests and BeautifulSoup. Below is a step-by-step guide, including 
code snippets and deployment suggestions.

Key Technologies:
Flask (web framework)
Requests (HTTP requests)
BeautifulSoup (HTML parsing)
SQLAlchemy/SQLite (optional, for storing scraped data)
Cloud deployment (Heroku, PythonAnywhere, etc.)
'''

# Step 1: Install Required Libraries

pip install flask requests beautifulsoup4 sqlalchemy


# Step 2: Basic Flask App Structure

from flask import Flask, jsonify, render_template
from bs4 import BeautifulSoup
import requests

app = Flask(__name__)

@app.route('/scrape')
def scrape():
    url = 'https://www.example.com'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    title = soup.find('title').text
    return jsonify({'title': title})

if __name__ == '__main__':
    app.run(debug=True)

'''
Step 3: Scraping Multiple Websites
To scrape multiple sites (e.g., YouTube, Amazon), you can define separate functions or endpoints for each, 
handling their unique HTML structures. Note: Both YouTube and Amazon restrict automated scraping; 
scraping them may violate their terms of service and often requires advanced techniques (e.g., headers, login, or using their APIs).
'''

@app.route('/scrape_all')
def scrape_all():
    urls = [
        {'name': 'Example', 'url': 'https://www.example.com'},
        # Add more URLs here
    ]
    results = []
    for site in urls:
        try:
            resp = requests.get(site['url'])
            soup = BeautifulSoup(resp.content, 'html.parser')
            title = soup.find('title').text
            results.append({'site': site['name'], 'title': title})
        except Exception as e:
            results.append({'site': site['name'], 'error': str(e)})
    return jsonify(results)

# Step 4: Display Data on Output Pages

from flask import render_template

@app.route('/scrape_display')
def scrape_display():
    url = 'https://www.example.com'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    title = soup.find('title').text
    return render_template('display.html', title=title)

'''
Step 5: Deploying on a Cloud Platform
        Popular Flask Hosting Providers:
        Heroku (easy, widely used for Flask apps)
        PythonAnywhere
        Google App Engine
        Railway.app
        Digital Ocean, Vercel, and others
    Heroku Deployment Steps (Summary):
        Create a requirements.txt (list your dependencies).
        Add a Procfile with: web: gunicorn app:app
        Initialize git, commit your code, and push to Heroku.
        Set environment variables as needed.
'''

# Example Directory Structure

/your-flask-app
    /templates
        display.html
    app.py
    requirements.txt
    Procfile


In [ ]:
# 2. Create a Flask app that consumes data from external APIs and displays it to users.
# Try to find an public API which will give you a data and based on that call it and deploy it on cloud platform.

'''
a Flask app that fetches data from a public API and displays it to users. Below is a practical example using the 
Open Library Books API, which is free, requires no authentication, and provides information about millions of books.
'''
# Step 1: Create the Flask App
# code for your Flask app (app.py):

pip install flask requests

from flask import Flask, render_template, request
import requests

app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def index():
    book_data = None
    if request.method == 'POST':
        isbn = request.form['isbn']
        url = f'https://openlibrary.org/api/books?bibkeys=ISBN:{isbn}&format=json&jscmd=data'
        response = requests.get(url)
        data = response.json()
        key = f'ISBN:{isbn}'
        if key in data:
            book_data = data[key]
    return render_template('index.html', book=book_data)

if __name__ == '__main__':
    app.run(debug=True)

'''
a simple HTML template (templates/index.html):

<!doctype html>
<html>
<head><title>Book Lookup</title></head>
<body>
  <h1>Open Library Book Lookup</h1>
  <form method="post">
    <input type="text" name="isbn" placeholder="Enter ISBN" required>
    <button type="submit">Search</button>
  </form>
  {% if book %}
    <h2>{{ book.title }}</h2>
    <p><strong>Authors:</strong> {{ book.authors|map(attribute='name')|join(', ') }}</p>
    <p><strong>Published:</strong> {{ book.publish_date }}</p>
  {% endif %}
</body>
</html>


Step 3: Deploying to a Cloud Platform (Heroku Example)
Prepare for Deployment:
Add a requirements.txt with your dependencies (flask, requests).
Add a Procfile with the line: web: gunicorn app:app.

Set Up Git and Heroku:
Initialize git: git init
Commit your code.
Create a Heroku app: heroku create
Push to Heroku: git push heroku master

Visit your deployed URL to use your app.

Best Practices
Use the requests library to fetch data from APIs.
Handle errors (e.g., invalid ISBN, API downtime).
Never hardcode sensitive information (for APIs that require keys, use environment variables).
Respect API rate limits.
'''

In [ ]:
# 3. Implement OAuth2 authentication to allow users to log in using their Google or Facebook accounts.

# Key Steps for OAuth2 Login

# Register Your App

# For Google: Register app in the Google Developer Console, create OAuth 2.0 credentials, and set your redirect URI.
# For Facebook: Register app at Facebook for Developers, create an app, and set up OAuth redirect URIs.

# Configure Flask App for OAuth2
# Here’s a minimal example using Authlib for Google login:
from flask import Flask, redirect, url_for, session
from authlib.integrations.flask_client import OAuth

app = Flask(__name__)
app.secret_key = 'YOUR_SECRET_KEY'
oauth = OAuth(app)

# Register Google OAuth
google = oauth.register(
    name='google',
    client_id='YOUR_GOOGLE_CLIENT_ID',
    client_secret='YOUR_GOOGLE_CLIENT_SECRET',
    access_token_url='https://oauth2.googleapis.com/token',
    access_token_params=None,
    authorize_url='https://accounts.google.com/o/oauth2/auth',
    authorize_params=None,
    api_base_url='https://www.googleapis.com/oauth2/v1/',
    client_kwargs={'scope': 'openid email profile'},
)

@app.route('/')
def homepage():
    user = dict(session).get('user', None)
    return f'Hello, {user["name"]}' if user else '<a href="/login">Login with Google</a>'

@app.route('/login')
def login():
    redirect_uri = url_for('authorize', _external=True)
    return google.authorize_redirect(redirect_uri)

@app.route('/authorize')
def authorize():
    token = google.authorize_access_token()
    resp = google.get('userinfo')
    user_info = resp.json()
    session['user'] = user_info
    return redirect('/')

@app.route('/logout')
def logout():
    session.pop('user', None)
    return redirect('/')

if __name__ == '__main__':
    app.run(debug=True)


'''
How OAuth2 Works in Flask
The user clicks "Login with Google" and is redirected to Google's OAuth consent screen.
After consenting, Google redirects the user back to your app with an authorization code.
Your app exchanges this code for an access token and retrieves user profile info.
The user’s info is stored in the session for authentication.

Deployment
Before deploying, set your production redirect URIs in the Google/Facebook developer consoles.
Deploy your Flask app to a cloud platform (Heroku, PythonAnywhere, etc.).
Ensure your client_id, client_secret, and secret_key are stored securely (use environment variables in production).
'''

In [ ]:
# 4. Develop a recommendation system using Flask that suggests content to users based on their preferences.

# Key Steps to Build the System
# Build the Recommendation Model
# Content-based filtering (recommends items similar to those the user liked, using item features)
# Collaborative filtering (recommends items liked by similar users)
# Hybrid approaches (combine both)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['description'])  # or any relevant text field
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

indices = pd.Series(df.index, index=df['title']).drop_duplicates()

def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # Top 10
    movie_indices = [i[0] for i in sim_scores]
    return df['title'].iloc[movie_indices]

# Create the Flask Web Application
# Set up a basic Flask app to interact with users and display recommendations:

from flask import Flask, render_template, request

app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def recommend():
    recommendations = []
    if request.method == 'POST':
        user_input = request.form['item_name']
        if user_input in df['title'].values:
            recommendations = get_recommendations(user_input)
        else:
            recommendations = ['Item not found.']
    return render_template('index.html', recommendations=recommendations)
